# Container Testing

Date: 5.7.21

Summary: Test the container class with the following characteristics:

- FCFS Queue
- Returning latency feedback to users
- Queues carry on between small timesteps
- Queues flush and reset between big timesteps
- Compatibility with central controller deployment.

In [1]:
cd /home/tkim/Academics/DistMEC

/home/tkim/Academics/DistMEC


Import relevant libraries.

In [2]:
import numpy as np
import math

from classes.Application import *
from classes.User import *
from classes.Server import *
from solver.Sim_Params import *
import copy
# from classes.Central_Controller import *

### Container Class

Build the class here and import to .py file.

In [3]:
import numpy as np

class Application:
    """
    Job: Associated with each user id and define
    - Job type, resource requirements, UE requirements, arrival, departure times
    """
    
    def __init__(self, job_type, user_id, time_steps, job_profiles):
        """
        job_type - integer [0,1,2] based on the sample profiles we have 
        user_id - associate job with user id
        """
        
        # Init values
        self.user_id = user_id
        self.job_type = job_type
        self.time_steps = time_steps
        self.job_profile = job_profiles[job_type] # Load rate, latency restriction      
       
        # Load latency/offload values
        self.latency_req = self.job_profile.latency_req
        self.offload_mean = self.job_profile.offload_mean
        
        # Record total amount of load generated per ts
        self.load_history = np.zeros(time_steps)
        
        # Record Reinforcement learning values below (UCB, confidence range)
        
        
        # Keep information on where relevant VM is
        
    def new_load(self,t):
        """
        Return a load value for this timestep based on exponential distribution value
        This will be logged into the 
        """
        
        self.load_history[t] =  np.random.exponential(1/self.offload_mean)
        return

    def cmab_round(self, arm_idx, arm_info, t):
        """
        Run CMAB to select how much load to offload to each arm
        """
        
        return
        

class Job_Profile:
    """
    Make list of job profiles with
    - UE properties (how much to offload generated per small ts)
    - Length Properties
    - Resource Consumption
    """
    
    def __init__(self, job_name,
                    latency_req,
                    offload_mean):
        """
        Add job profile to list 
        """
        
        self.job_name = job_name
        self.latency_req = latency_req # milisecond
        self.offload_mean = offload_mean

In [4]:
import numpy as np

class Container:
    """
    Container: Associated with a specific server-application pair for one-to-many matching
    Here 
    - Job type, resource requirements, UE requirements, arrival, departure times
    """
    
    def __init__(self, app_id, server_id, service_rate, latency_restriction):
        """
        app_id - which application this container is for
        server_id - which server this VM is designated for (binary on off)
        """
        
        self.app_id = app_id
        self.server_id = server_id
        self.deployed = False # True when active at server
        self.service_rate = service_rate
        self.latency_restriction = latency_restriction
        
        # queue --> [user_id, ts_arrive, load, remaining load]
        self.queue = np.empty((0,4))
        # history --> [user_id,ts_arrive,load,completion_time]
        self.history = np.empty((0,4))
        
    def add_to_queue(self, new_offload):
        """
        At the start of small TS add all queues
        new_offload -> np array of shape (1,3)
        """
        
        num_jobs = new_offload.shape[0]
        
        # Add new arrival information
        self.queue = np.append(self.queue,new_offload,axis=0)
        # Compute run time for each job
        loads = self.queue[:,3]
        load_cm = np.cumsum(loads)
        service_time = (load_cm/self.service_rate)[-num_jobs:]
        
        # Add to history
        new_history = copy.deepcopy(new_offload)
        new_history[:,3] = service_time
        
        self.history = np.append(self.history, new_history,axis=0)
        
        service_time_log = np.append(new_history[:,0].reshape(new_history[:,0].shape[0],1),
                                     service_time.reshape(service_time.shape[0],1),axis=1)
        
        return service_time_log
    
    def serve_ts(self):
        """
        subtract from queue based on the existing service rate
        Update self.queue and whatever remains
        """
        
        remaining_service = copy.deepcopy(self.service_rate)
        while remaining_service > 0:
            remainder = self.queue[0,3]
            if remainder <= remaining_service:
                self.queue = np.delete(self.queue,0,0)
            elif remainder > remaining_service:
                self.queue[0,3] = remainder - remaining_service
            
            remaining_service -= remainder
        
        return
        
    def calc_emp_beta(self):
        """
        Calculate the emprical value of beta based on latency violations
        """
        
        return

### Test Scenarios

1. generate containers, add to queue from user, process queue and return information to user


In [7]:
# Generate container and add jobs
j = 5

a = Container(app_id = 1, server_id = 1, service_rate = 10, latency_restriction=1)

offload = np.empty((0,4))

for i in range(j):
    load = np.random.randint(1,5,1)
    new_offload = np.array([[i,0,load,load]])
    offload = np.append(offload, new_offload, axis = 0)

np.random.shuffle(offload) 
    
val = a.add_to_queue(offload)
val

array([[4. , 0.2],
       [2. , 0.6],
       [3. , 0.9],
       [1. , 1. ],
       [0. , 1.4]])

In [10]:
a.queue

array([[0., 0., 4., 4.]])

In [9]:
a.serve_ts()

In [11]:
a.history

array([[4. , 0. , 2. , 0.2],
       [2. , 0. , 4. , 0.6],
       [3. , 0. , 3. , 0.9],
       [1. , 0. , 1. , 1. ],
       [0. , 0. , 4. , 1.4]])